In [0]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")


In [0]:
import pandas as pd
labeled_training_data_path = "https://raw.githubusercontent.com/jawdat23/Machine_Learning_Lab/master/word2vec-nlp-tutorial/labeledTrainData.tsv"
df = pd.read_csv(labeled_training_data_path, delimiter="\t")
df = df.drop(['id'], axis=1)
df.head()

In [9]:
import nltk.data
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text



In [0]:
df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

In [12]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

128.51009333333334

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D, Conv2D, GlobalMaxPool2D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
# model.add(Conv2D(filters=8, kernel_size=(3,3), activation='relu' , padding='same'))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 15


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 60000 samples, validate on 15000 samples
Epoch 1/15
60000/60000 [==============================] - 218s 4ms/step - loss: 0.3421 - acc: 0.8429 - val_loss: 0.2378 - val_acc: 0.9171
Epoch 2/15
60000/60000 [==============================] - 219s 4ms/step - loss: 0.2194 - acc: 0.9154 - val_loss: 0.2136 - val_acc: 0.9237
Epoch 3/15
60000/60000 [==============================] - 219s 4ms/step - loss: 0.1689 - acc: 0.9377 - val_loss: 0.1301 - val_acc: 0.9599
Epoch 4/15
60000/60000 [==============================] - 218s 4ms/step - loss: 0.1230 - acc: 0.9583 - val_loss: 0.0887 - val_acc: 0.9755
Epoch 5/15
60000/60000 [==============================] - 220s 4ms/step - loss: 0.0845 - acc: 0.9734 - val_loss: 0.0484 - val_acc: 0.9864
Epoch 6/15
60000/60000 [==============================] - 219s 4ms/step - loss: 0.0580 - acc: 0.9834 - val_loss: 0.0365 - val_acc: 0.9

In [0]:
testing_data_path = "https://raw.githubusercontent.com/jawdat23/Machine_Learning_Lab/master/word2vec-nlp-tutorial/testData.tsv"
df_test=pd.read_csv(testing_data_path, header=0, delimiter="\t", quoting=3)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x: clean_text(x))
list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = prediction > 0.5

In [0]:
y_sent = []
for y in y_pred:
  y_sent.append(int(y))
df_test["sentiment"] = y_sent
output = pd.DataFrame( data={"id":df_test["id"], "sentiment":df_test["sentiment"]} )
output.to_csv( "DL_LSTM_15.csv", index=False, quoting=3 )